<a href="https://colab.research.google.com/github/prinaldi3/Denoising/blob/main/Gaussian_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!python -m pip install pip==21.0.1

In [ ]:
!rm -rf xca
!git clone https://github.com/maffettone/xca

In [ ]:
%%bash
cd xca
python -m pip install .

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xca

In [ ]:
#Function to generate guassian 

def guassian(x, mu, sig):
  return (1 / (sig * np.sqrt(2*np.pi))) * np.exp(-1/2 * ((x - mu)/sig)**2)


In [ ]:
from random import random
from scipy.interpolate import UnivariateSpline

#Generate dataset

#Hyperparams 
x_min = -10
x_max = 10
n_points = 500
n_samples = 100
x = np.linspace(x_min, x_max, n_points)
pixel_width = (x_max - x_min) / n_points

fwhm = 0 #full width half max
y = [] #this will ultimately be the data

for i in range(n_samples):
  #Compute random centers that are sufficiently seperated (i.e. on either sides of the origin)
  mu_1 = (x_min + 1) + random() * ( 0 - (x_min + 1) )
  mu_2 = random() * (x_max - 1)

  #Compute generally narrow spreads -- adjust as needed to control FWHM
  sig_1 = np.random.uniform(.42, .46)
  sig_2 = np.random.uniform(.42, .46)

  y_1 = guassian(x, mu_1, sig_1)
  y_2 = guassian(x, mu_2, sig_2)

  #Calculate Full Width Half Max of each peak, append to list for later averaging
  spline_1 = UnivariateSpline(x, y_1-np.max(y_1)/2, s=0)
  r1_1, r2_1 = spline_1.roots()
  fwhm += abs(r2_1 - r1_1)

  spline_2 = UnivariateSpline(x, y_2 - np.max(y_2)/2, s=0)
  r1_2, r2_2 = spline_2.roots()
  fwhm += abs(r2_2 - r1_2)

  sample = y_1 + y_2
  y.append(sample)

y = np.array(y)
avg_fwhm = fwhm / (2*n_samples) #two guassians per sample, so it's 2*n_samples
avg_fwhm = int(avg_fwhm/pixel_width)
print("Average FWHM: {}".format(avg_fwhm))

In [ ]:
#Example plot of guassians

fig, ax = plt.subplots()
plt.rc('font', size=20)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
ax.plot(x, y[1])
ax.set_title("Average FWHM={} Pixels".format(avg_fwhm))
ax.set_xlabel("Pixels")
ax.set_ylabel("Intensity [Arb.]")
#fig.savefig("/content/gdrive/MyDrive/Final_Plots/guassian_sample_fwhm_{}".format(avg_fwhm), facecolor="white")

In [ ]:
#Save the data to .npy

from pathlib import Path
for i in range(len(y)):
  np.save('change_this_path_as_needed'.format(i), y[i])

In [ ]:
from xca.ml.tf_data_proc import np_dir_to_record

#Make tfrecords file 
npy_path = Path('change_this_path_as_needed')
tfr_path = Path('change_this_path_as_needed')
np_dir_to_record(npy_path=npy_path, tfr_path=tfr_path)
